In [ ]:
from pyld import jsonld
from urllib.parse import urlparse
import os 
from urllib.parse import urlparse, urlunparse, urljoin

# jsonld.set_document_loader(jsonld.aiohttp_document_loader(timeout=...))

In [ ]:

def get_relative_path(base_url, target_url):
    """
    Computes the relative path from a base URL to a target URL.
    
    Parameters:
    - base_url (str): The base URL.
    - target_url (str): The target URL.
    
    Returns:
    - str: The relative path from the base URL to the target URL.
    """
    # Parse URLs
    base_parsed = urlparse(base_url)
    target_parsed = urlparse(target_url)

    # Ensure both URLs share the same scheme and netloc
    if base_parsed.scheme != target_parsed.scheme or base_parsed.netloc != target_parsed.netloc:
        raise ValueError("Both URLs must have the same scheme and domain (netloc).")
    
    # Compute relative path using os.path.relpath
    base_path = base_parsed.path
    target_path = target_parsed.path

    # Normalize base path to ensure directory-like behavior
    if not base_path.endswith('/'):
        base_path = os.path.dirname(base_path) + '/'

    relative_path = os.path.relpath(target_path, start=base_path)

    return relative_path

In [ ]:

def local_document_loader(base_local_path):
    """
    Returns a custom document loader that rewrites URLs to local paths.
    
    Parameters:
    - base_local_path (str): Base directory path for local resources.

    Returns:
    - Callable: A custom document loader function.
    """
    original_loader = jsonld.requests_document_loader()

    def loader(url,options):
        # Rewrite the URL to a local path
        print(options)
        # parsed_url = urlparse(url)
        # local_path = f"{base_local_path}{parsed_url.path}"
        local_path = get_relative_path(base_local_path, url)
        # print(parsed_url)

        # Read the local file instead of fetching the URL
        try:
            with open('./data_descriptors/'+local_path, 'r', encoding='utf-8') as file:
                local_data = file.read()
            return {
                "contextUrl": None,  # Remote context URLs aren't rewritten here
                "documentUrl": url,  # Keep the original document URL for reference
                "document": local_data
            }
        except FileNotFoundError:
            raise RuntimeError(f"Local path not found for {url}: {local_path}")
        # original_loader(url,options)

    return loader



In [ ]:
repo = os.popen("git remote get-url origin").read().replace('.git','').strip('\n').split('/')[-2:]
base_local_path = f'https://{repo[0].lower()}.github.io/{repo[1]}/'

# Set the custom document loader
loader = local_document_loader(base_local_path)
jsonld.set_document_loader(loader)

base_local_path


In [ ]:
# Expand the JSON-LD document
expanded = jsonld.expand('https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/atmos.json',options={'documentLoader': loader})
print(expanded)

In [ ]:
local_document_loader('./')

In [ ]:
get_relative_path(base_local_path,'https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/atmos.json')

In [2]:
# import aiohttp
# import asyncio
# from pyld import jsonld
# # !pip install aiohttp
# jsonld.set_document_loader(jsonld.aiohttp_document_loader(timeout=10))
# asyncio.run(jsonld.expand('./data_descriptors/realm/atmos.json'))

# # ,options={'documentLoader': jsonld.aiohttp_document_loader(timeout=10)}))

# asyncio.run(jsonld.expand('./data_descriptors/realm/atmos.json'))